first,we'll parse the robots.txt to avoid downlaoding blocked URLs

urllib is sooo cool , it comes with robotparser

In [1]:
from urllib import robotparser

rp = robotparser.RobotFileParser()
rp.set_url('https://www.example.com/robots.txt')
rp.read()
url = 'https://www.example.com'
user_agent ='BadCrawler'
rp.can_fetch(user_agent,url)


True

the above code gives whether a particular webcrawler can access the webpage or not


In [1]:
def get_robots_parser(robots_url):
    " Return the robots parser object using the robots_url "
    rp = robotparser.RobotFileParser()
    rp.set_url(robots_url)
    rp.read()
    return rp

Here, we define the get_robots_parser, which return the robot_parser

In [2]:
def link_crawler(start_url, link_regex, robots_url=None, user_agent='wswp'):
    ...
    if not robots_url:
        robots_url = '{}/robots.txt'.format(start_url)
    rp = get_robots_parser(robots_url)

SyntaxError: invalid syntax (<ipython-input-2-bebbc333756b>, line 2)

In [3]:
def download(url,user_agent='naruto',number_retries = 2, charset='utf-8'):
    print("Downloading:", url)
    request = urllib.request.Request(url)
    request.add_header("User-agent", user_agent)
    try:
        resp = urllib.request.urlopen(request)
        #verify if web charset is utf-8, if not ,we'll use utf-8 to decode, haha, 
        #it will throw error if no cs returned, but
        #we hope utf-8 will help
        #read more on pypi.python.org/pypi/chardet to implement a more robust decoder
        cs = resp.headers.get_content_charset()
        if not cs:
            cs = charset
        html = resp.read().decode(cs)
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error:', e.reason)
        html = None
        if number_retries > 0:
            if hasattr(e, "code") and 500 <= e.code <= 600:
                #recursive download process
                return download(url,number_retries -1)
    return html

def get_robots_parser(robots_url):
    " Return the robots parser object using the robots_url "
    rp = robotparser.RobotFileParser()
    rp.set_url(robots_url)
    rp.read()
    return rp

def link_crawler(start_url, link_regex, robots_url=None, user_agent='naruto'):
    
    """
    we'll start from a base url and if the link match the link_regex,
    we'll continue the crawler process
    """
    if not robots_url:
        robots_url = '{}/robots.txt'.format(start_url)
    rp = get_robots_parser(robots_url)  
    
    crawl_queue = [start_url]
    seen = set(crawl_queue)
    while crawl_queue:
        url = crawl_queue.pop()
        if rp.can_fetch(user_agent,url):
            html = download(url,user_agent=user_agent)
            if not html:
                continue
            #the following code is to get the type of links we want
            for link in get_links(html):
                if re.match(link_regex, link):
                    abs_link = urljoin(start_url, link)
                    #let's check whether link is in crawl_queue
                    if abs_link not in seen:
                        seen.add(abs_link)
                        crawl_queue.append(abs_link)
        else:
            print("Blocked By robots.txt:", url)
            
    